In [7]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random

N = 400
feats = 784
D = (rng.randn(N, feats).astype(theano.config.floatX),
rng.randint(size=N,low=0, high=2).astype(theano.config.floatX))
training_steps = 10000

# Configuration
theano.config.floatX = 'float32'

# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")
w = theano.shared(rng.randn(feats).astype(theano.config.floatX), name="w")
b = theano.shared(numpy.asarray(0., dtype=theano.config.floatX), name="b")
x.tag.test_value = D[0]
y.tag.test_value = D[1]
#print "Initial model:"
#print w.get_value(), b.get_value()

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w)-b)) # Probability of having a one
prediction = p_1 > 0.5 # The prediction that is done: 0 or 1
xent = -y*T.log(p_1) - (1-y)*T.log(1-p_1) # Cross-entropy
cost = T.cast(xent.mean(), 'float32') + 0.01*(w**2).sum() # The cost to optimize
gw,gb = T.grad(cost, [w,b])

# Compile expressions to functions
train = theano.function(
            inputs=[x,y],
            outputs=[prediction, xent],
            updates={w:w-0.01*gw, b:b-0.01*gb},
            name = "train")
predict = theano.function(inputs=[x], outputs=prediction,
            name = "predict")

if any([x.op.__class__.__name__ in ['Gemv', 'CGemv', 'Gemm', 'CGemm'] for x in
        train.maker.fgraph.toposort()]):
    print 'Used the cpu'
elif any([x.op.__class__.__name__ in ['GpuGemm', 'GpuGemv'] for x in
          train.maker.fgraph.toposort()]):
    print 'Used the gpu'
else:
    print 'ERROR, not able to tell if theano used the cpu or the gpu'
    print train.maker.fgraph.toposort()

for i in range(training_steps):
    pred, err = train(D[0], D[1])
#print "Final model:"
#print w.get_value(), b.get_value()

print "target values for D"
print D[1]

print "prediction on D"
print predict(D[0])

Used the cpu
target values for D
[ 1.  0.  0.  0.  0.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  0.  0.  0.
  1.  1.  0.  1.  1.  0.  1.  1.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.
  1.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  0.  0.
  0.  1.  1.  1.  1.  0.  1.  0.  1.  0.  1.  0.  0.  1.  1.  0.  0.  1.
  1.  1.  0.  0.  1.  0.  0.  1.  0.  1.  0.  0.  1.  0.  0.  0.  1.  0.
  1.  1.  0.  0.  1.  0.  1.  1.  1.  0.  1.  0.  0.  0.  1.  0.  1.  0.
  1.  1.  1.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  1.  1.  0.  0.  1.
  1.  1.  1.  0.  0.  1.  0.  0.  0.  1.  0.  1.  1.  0.  0.  0.  1.  1.
  0.  1.  1.  1.  1.  0.  1.  1.  0.  1.  0.  1.  0.  1.  1.  0.  1.  0.
  1.  1.  1.  0.  0.  0.  0.  0.  1.  1.  0.  1.  0.  0.  1.  1.  1.  0.
  1.  1.  1.  1.  1.  0.  1.  1.  0.  1.  1.  0.  1.  0.  1.  1.  0.  1.
  0.  1.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  0.  1.  1.
  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  0.  1.  0.  0.  1.  1.  1.
  1.  0.  0.  1.  

C:\SciSoft\WinPython-32bit-2.7.9.4\python-2.7.9\lib\site-packages\IPython\kernel\__main__.py:37: UserWarning: The parameter 'updates' of theano.function() expects an OrderedDict, got <type 'dict'>. Using a standard dictionary here results in non-deterministic behavior. You should use an OrderedDict if you are using Python 2.7 (theano.compat.python2x.OrderedDict for older python), or use a list of (shared, update) pairs. Do not just convert your dictionary to this type before the call as the conversion will still be non-deterministic.
